In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import mannwhitneyu, ttest_ind

In [3]:
# Seed for reproducibility
np.random.seed(42)

In [4]:

def generate_dataset():
    turma_a = np.random.normal(75, 10, 125).clip(0, 100)  # Scores: mean=75, std=10
    turma_b = np.random.normal(70, 15, 125).clip(0, 100)  # Scores: mean=70, std=15
    df = pd.DataFrame({
        "Aluno": [f"Aluno_{i}" for i in range(1, 251)],
        "Turma": ["A"] * 125 + ["B"] * 125,
        "Nota": np.concatenate([turma_a, turma_b])
    })
    return df

In [5]:
# Step 1: Generate and save the dataset
data = generate_dataset()
data.to_csv("notas_escola.csv", index=False)

In [6]:
# Step 4: Hypotheses explanation and output interpretation
hypotheses = """
### Hypotheses:
1. **Null Hypothesis (H0):**
   - t-test: The mean scores of Turma A and Turma B are equal (μ_A = μ_B).
   - U-test: The distributions of scores for Turma A and Turma B are identical (F_A(x) = F_B(x)).

2. **Alternative Hypothesis (H1):**
   - t-test: The mean scores of Turma A and Turma B are different (μ_A ≠ μ_B).
   - U-test: The distributions of scores for Turma A and Turma B are different (F_A(x) ≠ F_B(x)).

### Interpretation:
- **If p-value ≤ 0.05:** Reject the null hypothesis, indicating a statistically significant difference.
- **If p-value > 0.05:** Fail to reject the null hypothesis, indicating no statistically significant difference.

### Contextual Example:
- Turma A: Mean-centered teaching.
- Turma B: Problem-based learning.
The test helps assess if teaching methods have a statistically significant impact on student performance.
"""

In [7]:
# Step 2: Define hypothesis test functions
# Shapiro-Wilk normality test
def e_normal(group):
    wval, pval = stats.shapiro(group)
    if len(group) > 5000:
        if wval < 0.05:
            return False
    if pval < 0.05:
        return False
    return True

# Levene's test for equal variances
def var_igual(group1, group2):
    _, pval = stats.levene(group1, group2)
    if pval < 0.05:
        return False
    return True

# Hypothesis testing
def stats_test(group1, group2):
    if e_normal(group1) and e_normal(group2):
        if var_igual(group1, group2):
            print("Realizando t-test...")
            return stats.ttest_ind(group1, group2)
        print("Realizando t-test com variancia diferente...")
        return stats.ttest_ind(group1, group2, equal_var=False)
    print("Realizando U-Test...")
    return mannwhitneyu(group1, group2)

In [8]:
# Step 3: Load dataset and perform hypothesis testing
csv_path = "notas_escola.csv"
df = pd.read_csv(csv_path)
group_a = df[df["Turma"] == "A"]["Nota"].values
group_b = df[df["Turma"] == "B"]["Nota"].values

print("\n=== Resultado do Teste de Hipótese ===")
resultado = stats_test(group_a, group_b)
print(resultado)
print(hypotheses)


=== Resultado do Teste de Hipótese ===
Realizando t-test com variancia diferente...
TtestResult(statistic=2.4379918688720497, pvalue=0.0155751883447904, df=216.56923552345978)

### Hypotheses:
1. **Null Hypothesis (H0):**
   - t-test: The mean scores of Turma A and Turma B are equal (μ_A = μ_B).
   - U-test: The distributions of scores for Turma A and Turma B are identical (F_A(x) = F_B(x)).

2. **Alternative Hypothesis (H1):**
   - t-test: The mean scores of Turma A and Turma B are different (μ_A ≠ μ_B).
   - U-test: The distributions of scores for Turma A and Turma B are different (F_A(x) ≠ F_B(x)).

### Interpretation:
- **If p-value ≤ 0.05:** Reject the null hypothesis, indicating a statistically significant difference.
- **If p-value > 0.05:** Fail to reject the null hypothesis, indicating no statistically significant difference.

### Contextual Example:
- Turma A: Mean-centered teaching.
- Turma B: Problem-based learning.
The test helps assess if teaching methods have a statisti

In [13]:
# Teste de proporção para o grupo 1 (Turma A)
from statsmodels.stats.proportion import proportions_ztest

nota = 90

# Parâmetros do teste
p_esperado = 0.3  # Proporção esperada
n_total = len(group_a)  # Tamanho da amostra
n_sucessos = (group_a >= nota).sum()  # Número de alunos com nota > nota

# Estatística z e p-valor
stat, p_valor = proportions_ztest(n_sucessos, n_total, value=p_esperado)

# Resultados
print(f"Estatística z: {stat:.4f}")
print(f"p-valor: {p_valor:.4f}")

# Interpretação
if p_valor <= 0.05:
    print(f'Rejeitamos H₀: A proporção de alunos com notas superiores a {nota} é significativamente diferente de 30%.')
else:
    print(f'Falhamos em rejeitar H₀: A proporção de alunos com notas superiores a {nota} não é significativamente diferente de {p_esperado *100 :.2f}%.')

Estatística z: -11.8649
p-valor: 0.0000
Rejeitamos H₀: A proporção de alunos com notas superiores a 90 é significativamente diferente de 30%.
